In [ ]:
import sys


if sys.version.split()[0] != '3.10.12':
    print(sys.version)
    raise RuntimeError('Wrong Python version. Go to ENVIRONMENT settings, Stop machine, Start machine')


import tensorflow
if tensorflow.__version__ != '2.13.0':
    raise RuntimeError('Wrong TF version. Go to ENVIRONMENT settings, Stop machine, Start machine')


print('\nPython version: OK')
print('TensorFlow version: OK')

2024-01-02 14:46:23.029029: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-02 14:46:23.030827: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-02 14:46:23.065978: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-02 14:46:23.067010: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 14:46:24.454343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.020,
    'frame_step_in_s': 0.010,
    'num_mel_bins': 10,
    'lower_frequency': 20,
    'upper_frequency': 4000,
    'num_coefficients': 40,
}

TRAINING_ARGS = {
    'batch_size': 30,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 20
}
final_sparsity = 0.70
alpha = 0.25

In [ ]:
import tensorflow as tf
yes_no_train_ds = tf.data.Dataset.list_files('/tmp/yn-train/*')
yes_no_test_ds = tf.data.Dataset.list_files('/tmp/yn-test/*')

In [ ]:
num_train_files = len(yes_no_train_ds)
num_test_files = len(yes_no_test_ds)

print('Training set size:', num_train_files)
print('Test set size:', num_test_files)

Training set size: 1600
Test set size: 200


In [ ]:
from preprocessing import LABELS
from preprocessing import AudioReader
from preprocessing import MelSpectrogram
from preprocessing import MFCC

audio_reader = AudioReader(tf.int16, 16000)
mfcc_processor = MFCC(**PREPROCESSING_ARGS)

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id



batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = (yes_no_train_ds
            .map(audio_reader.get_audio_and_label)
            .map(mfcc_processor.get_mfccs_and_label)
            .map(prepare_for_training)
            .batch(batch_size)
            .cache())
test_ds = (yes_no_test_ds
            .map(audio_reader.get_audio_and_label)
            .map(mfcc_processor.get_mfccs_and_label)
            .map(prepare_for_training)
            .batch(batch_size))

In [ ]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (30, 99, 10, 1)
Data Shape: (99, 10, 1)
Labels: tf.Tensor([1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1], shape=(30,), dtype=int64)
2024-01-02 15:07:31.412715: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# FINAL MODEL

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=32, kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

# SETUP MAGNITUDE BASED WEIGHTS PRUNING

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epochs * 0.2)
end_step = int(len(train_ds) * epochs)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

In [ ]:
model_for_pruning.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 49, 4, 32)         578       
 _10 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_batch_  (None, 49, 4, 32)         129       
 normalization_10 (PruneLow                                      
 Magnitude)                                                      
                                                                 
 prune_low_magnitude_re_lu_  (None, 49, 4, 32)         1         
 10 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2d  (None, 49, 4, 16)         9218      
 _11 (PruneLowMagnitude)                                         
                                                      

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epochs, callbacks=callbacks)

Epoch 1/20
54/54 [==============================] - 18s 283ms/step - loss: 0.3898 - sparse_categorical_accuracy: 0.8425
Epoch 2/20
54/54 [==============================] - 1s 22ms/step - loss: 0.2318 - sparse_categorical_accuracy: 0.9181
Epoch 3/20
54/54 [==============================] - 1s 22ms/step - loss: 0.1637 - sparse_categorical_accuracy: 0.9406
Epoch 4/20
54/54 [==============================] - 1s 21ms/step - loss: 0.1326 - sparse_categorical_accuracy: 0.9506
Epoch 5/20
54/54 [==============================] - 1s 21ms/step - loss: 0.1029 - sparse_categorical_accuracy: 0.9650
Epoch 6/20
54/54 [==============================] - 1s 21ms/step - loss: 0.0857 - sparse_categorical_accuracy: 0.9719
Epoch 7/20
54/54 [==============================] - 1s 21ms/step - loss: 0.0704 - sparse_categorical_accuracy: 0.9762
Epoch 8/20
54/54 [==============================] - 1s 21ms/step - loss: 0.0531 - sparse_categorical_accuracy: 0.9850
Epoch 9/20
54/54 [==============================] - 1s

In [ ]:
import numpy as np


for layer in model_for_pruning.layers:
    if isinstance(layer, tf.keras.layers.Wrapper):
        weights = layer.trainable_weights
    else:
        weights = layer.weights
    for weight in weights:        
        weight_size = weight.numpy().size
        zero_num = np.count_nonzero(weight == 0)
        print(
            f'{weight.name}: {zero_num/weight_size:.2%} sparsity ',
            f'({zero_num}/{weight_size})',
        )

conv2d_10/kernel:0: 70.14% sparsity  (202/288)
batch_normalization_10/gamma:0: 0.00% sparsity  (0/32)
batch_normalization_10/beta:0: 0.00% sparsity  (0/32)
conv2d_11/kernel:0: 69.99% sparsity  (3225/4608)
batch_normalization_11/gamma:0: 0.00% sparsity  (0/16)
batch_normalization_11/beta:0: 0.00% sparsity  (0/16)
conv2d_12/kernel:0: 69.97% sparsity  (1612/2304)
batch_normalization_12/gamma:0: 0.00% sparsity  (0/16)
batch_normalization_12/beta:0: 0.00% sparsity  (0/16)
conv2d_13/kernel:0: 69.99% sparsity  (3225/4608)
batch_normalization_13/gamma:0: 0.00% sparsity  (0/32)
batch_normalization_13/beta:0: 0.00% sparsity  (0/32)
dense_3/kernel:0: 70.31% sparsity  (45/64)
dense_3/bias:0: 0.00% sparsity  (0/2)


In [ ]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

7/7 [==============================] - 2s 265ms/step - loss: 0.0457 - sparse_categorical_accuracy: 0.9900


In [ ]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]


print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.0200
Training Accuracy: 99.62%

Test Loss: 0.0457
Test Accuracy: 99.00%


# save my model

In [ ]:
import os
from time import time

timestamp = int(time())

saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1704208216/assets
INFO:tensorflow:Assets written to: ./saved_models/1704208216/assets


In [ ]:
!ls saved_models

1703113800  1703194192	1703371682  1703896291	1704207759  ref_model
1703193655  1703255334	1703670719  1703938625	1704208216


# save hyperparameters and results

In [ ]:
import pandas as pd

output_dict = {
    'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'final_sparsity': final_sparsity,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])

output_path='./spectrogram_wp_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

# create tflite version of the model

In [ ]:
MODEL_NAME = "1704208216"
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
tflite_model = converter.convert()

2024-01-02 15:13:37.670539: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-01-02 15:13:37.670574: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-01-02 15:13:37.841842: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/1704208216
2024-01-02 15:13:38.014460: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-01-02 15:13:38.014495: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: ./saved_models/1704208216
2024-01-02 15:13:38.018879: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-01-02 15:13:38.435259: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: ./saved_models/1704208216
2024-01-02 15:13:38.453258: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status

In [ ]:
import os
tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

In [ ]:
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name

'./tflite_models/1704208216.tflite'

In [ ]:
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

# converting file into zip file

In [ ]:
import zipfile

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

# model size

In [ ]:

tflite_size = os.path.getsize(tflite_model_name) / 1024.0
zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0

print(f'Original tflite size (not pruned model): {tflite_size:.3f} KB')
print(f'Zipped tflite size (pruned model): {zipped_size:.3f} KB')


Original tflite size (not pruned model): 50.367 KB
Original tflite size (pruned model): 50.367 KB
Zipped tflite size (not pruned model): 20.093 KB
Zipped tflite size (pruned model): 20.093 KB


# latency

In [ ]:
import tensorflow as tf
import os
from preprocessing import MelSpectrogram
import numpy as np
from time import time

REF_PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000,
}

mel_spec_processor = MelSpectrogram(**REF_PREPROCESSING_ARGS)
interpreter = tf.lite.Interpreter(model_path='tflite_models/ref_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

audio = tf.random.normal((16000,))

ref_latencies = []

for i in range(100):
    start_preprocess = time()

    log_mel_spectrogram = mel_spec_processor.get_mel_spec(audio)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, 0)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)
    interpreter.set_tensor(input_details[0]['index'], log_mel_spectrogram)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    end_inference = time()

    ref_latencies.append(end_inference - start_preprocess)

median_ref_latency = np.median(ref_latencies)
median_ref1_latency = 1000* median_ref_latency
print(median_ref1_latency)

16.301393508911133


In [ ]:
import tensorflow as tf
import os
from preprocessing import MelSpectrogram
from preprocessing import MFCC
import numpy as np
from time import time

mfcc_processor = MFCC(**PREPROCESSING_ARGS)
interpreter = tf.lite.Interpreter(model_path=r'/work/tflite_models/1704208216.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
audio = tf.random.normal((16000,))
OPT4_latencies = []

for i in range(100):
    start_preprocess = time()

    mfcc_features = mfcc_processor.get_mfccs(audio)
        #mfcc_features = tf.reshape(mfcc_features, input_details[0]['shape'])
    mfcc_features = tf.expand_dims(mfcc_features, 0)  # Add the batch dimension
    mfcc_features = tf.expand_dims(mfcc_features, -1)
    interpreter.set_tensor(input_details[0]['index'],mfcc_features)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    end_inference = time()


    OPT4_latencies.append(end_inference - start_preprocess)

median_OPT4_latency = np.median(OPT4_latencies)
median_OPT4_latency = 1000*median_OPT4_latency
print(median_OPT4_latency)

8.942604064941406


In [ ]:
TOTAL_LATENCY_SAVING = 100 * (median_ref1_latency - median_OPT4_latency) / median_ref1_latency
print(TOTAL_LATENCY_SAVING)

45.142088251210275


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eef6af2f-fd81-4b1e-aa00-f5faf5eb94b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>